In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Configure IAM Policy in Vertex AI Feature Store


<table align="left">

  <td>
    <a href="TODO">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="TODO">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="TODO">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This tutorial demonstrates how to configure an IAM Policy to control access to resources and data stored within `Vertex AI Feature Store`.

Learn more about [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore/overview).

### Objective

In this tutorial, you will learn to use Cloud IAM and Vertex AI Feature Store to control access to resources and data used for online serving.

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Feature Store`
- `Cloud IAM`
- `BigQuery`

The steps performed include:
- Create a BigQuery dataset and set up a `FeatureView` to run periodic sync jobs.
- Provision an online store instance and set up online serving.
- Configure an IAM Policy to manage access controls.

### Costs

This tutorial uses billable components of Google Cloud:

* `Vertex AI`
* `BigQuery`

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and
[BigQuery pricing](https://cloud.google.com/bigquery/pricing)
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to run this notebook.

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform\
                                 google-cloud-bigquery

### Restart the kernel

After you install the SDK, you need to restart the notebook kernel so it can find the packages. Restart kernel from *Kernel -> Restart Kernel*, or by running the following:

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Authenticate your Google Cloud account
Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab**

In [ ]:
import os
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
elif not os.getenv("IS_TESTING"):
    %env GOOGLE_APPLICATION_CREDENTIALS ''

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Initial Setup

Import the following libraries to use Vertex AI Feature Store.

In [ ]:
from google.cloud import aiplatform, bigquery
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient, FeatureOnlineStoreServiceClient,
    FeatureRegistryServiceClient,)
from google.cloud.aiplatform_v1beta1.types import feature as feature_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_group as feature_group_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_online_store as feature_online_store_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_online_store_admin_service as \
    feature_online_store_admin_service_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_online_store_service as feature_online_store_service_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_registry_service as feature_registry_service_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_view as feature_view_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_selector as feature_selector_pb2
from google.iam.v1 import iam_policy_pb2
from google.iam.v1 import policy_pb2
from http import HTTPStatus

Specify the GCP Project ID.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Select a region to be used when constructing the the Vertex AI regional API endpoint. For example, `us-central1` corresponds to `us-central1-aiplatform.googleapis.com`.

<details>
<summary>View list of supported regions</summary>
<br>

- asia-southeast1
- europe-west4
- us-central1
- us-east1
- us-east4
- us-west1
- us-west3
- europe-west2
- asia-northeast2
- us-west2
- europe-west3
</details>

In [ ]:
REGION = "us-central1"  # @param {type:"string"}
LOCATION_PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

Configure the necessary service clients:
- [FeaturestoreServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.featurestore_service.FeaturestoreServiceClient)
- [FeatureOnlineStoreAdminServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_online_store_admin_service.FeatureOnlineStoreAdminServiceClient)
- [FeatureRegistryServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_registry_service.FeatureRegistryServiceClient)

In [ ]:
ENDPOINT = REGION + "-aiplatform.googleapis.com"
featurestore_service_client = FeaturestoreServiceClient(client_options={"api_endpoint": ENDPOINT})
feature_online_store_admin_service_client = FeatureOnlineStoreAdminServiceClient(client_options={"api_endpoint": ENDPOINT})
feature_registry_service_client = FeatureRegistryServiceClient(client_options={"api_endpoint": ENDPOINT})

Generate sample data for this colab.

In [ ]:
DATASET_ID = "test_data"+"_"+REGION.replace('-', '_')  # @param {type:"string"}
TABLE_ID = "tableA"  # @param {type:"string"}

!bq mk --dataset_id={DATASET_ID} --location={REGION}
!bq query --nouse_legacy_sql \
"CREATE TABLE {DATASET_ID}.{TABLE_ID} AS (" \
"SELECT * FROM UNNEST(ARRAY<STRUCT<entity_id STRING, feature_timestamp TIMESTAMP, feature1 INT64, feature2 INT64>>[" \
"('test', TIMESTAMP('2024-02-26 08:00:00 UTC'), 10, 20)," \
"('test', TIMESTAMP('2024-02-27 08:00:00 UTC'), 30, 40)," \
"('test', TIMESTAMP('2024-02-28 08:00:00 UTC'), 50, 60)]))"

## Configure Online Serving

Provision a `FeatureOnlineStore` instance.

In [ ]:
FEATURE_ONLINE_STORE_ID = "test_feature_online_store"
feature_online_store_admin_service_client.create_feature_online_store({
  "parent": LOCATION_PARENT,
  "feature_online_store_id": FEATURE_ONLINE_STORE_ID,
  "feature_online_store": {
    "bigtable": {
        "auto_scaling": {
            "min_node_count": 1,
            "max_node_count": 3,
            "cpu_utilization_target": 50
        }
    }
  }
}).result()

Configure periodic data synchronization from the offline store in BigQuery to the `FeatureOnlineStore` instance.

In [ ]:
FEATURE_VIEW_ID = "test_feature_view"
create_feature_view_result = feature_online_store_admin_service_client.create_feature_view(
    parent = LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID,
    feature_view_id = FEATURE_VIEW_ID,
    feature_view = {
        "big_query_source": {
          "uri": f"bq://{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}",
          "entity_id_columns": ["entity_id"]
        },
        "sync_config": { "cron": "0 12 * * *" }
    }
).result()

Configure IAM Policy for a feature online store.

In [ ]:
# A specific user email or a group email can be used to add members to the IAM Policy.
EXAMPLE_RESOURCE_VIEWER_EMAIL = "" # @param {type:"string"}
EXAMPLE_DATA_VIEWER_EMAIL = "" # @param {type:"string"}
# A Service Account can also be added to the IAM Policy.
EXAMPLE_ADMIN_SERVICE_ACCOUNT = "" # @param {type:"string"}
feature_online_store_admin_service_client.set_iam_policy(request = iam_policy_pb2.SetIamPolicyRequest(
    resource=LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID,
    policy = policy_pb2.Policy(
        bindings = [
          policy_pb2.Binding(
              role = "roles/aiplatform.featurestoreResourceViewer",
              members = [f"user:{EXAMPLE_RESOURCE_VIEWER_EMAIL}"]
          ),
          policy_pb2.Binding(
              role = "roles/aiplatform.featurestoreDataViewer",
              members = [f"user:{EXAMPLE_DATA_VIEWER_EMAIL}"]
          ),
          policy_pb2.Binding(
              role = "roles/aiplatform.admin",
              members = [f"serviceAccount:{EXAMPLE_ADMIN_SERVICE_ACCOUNT}"]
          )
        ]
    )
))

Confirm the updated IAM Policy of the feature online store.

In [ ]:
feature_online_store_admin_service_client.get_iam_policy(request = iam_policy_pb2.GetIamPolicyRequest(
    resource=LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID
))

Configure IAM Policy for a feature view.

In [ ]:
# A specific user email or a group email can be used to add members to the IAM Policy.
EXAMPLE_RESOURCE_VIEWER_EMAIL = "" # @param {type:"string"}
EXAMPLE_DATA_VIEWER_EMAIL = "" # @param {type:"string"}
# A Service Account can also be added to the IAM Policy.
EXAMPLE_ADMIN_SERVICE_ACCOUNT = "" # @param {type:"string"}
feature_online_store_admin_service_client.set_iam_policy(request = iam_policy_pb2.SetIamPolicyRequest(
    resource=LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID + "/featureViews/" + FEATURE_VIEW_ID,
    policy = policy_pb2.Policy(
        bindings = [
          policy_pb2.Binding(
              role = "roles/aiplatform.featurestoreResourceViewer",
              members = [f"user:{EXAMPLE_RESOURCE_VIEWER_EMAIL}"]
          ),
          policy_pb2.Binding(
              role = "roles/aiplatform.featurestoreDataViewer",
              members = [f"user:{EXAMPLE_DATA_VIEWER_EMAIL}"]
          ),
          policy_pb2.Binding(
              role = "roles/aiplatform.admin",
              members = [f"serviceAccount:{EXAMPLE_ADMIN_SERVICE_ACCOUNT}"]
          )
        ]
    )
))

Confirm the updated IAM Policy.

In [ ]:
feature_online_store_admin_service_client.get_iam_policy(request = iam_policy_pb2.GetIamPolicyRequest(
    resource=LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID + "/featureViews/" + FEATURE_VIEW_ID
))

Trigger a `FeatureViewSync`

In [ ]:
feature_online_store_admin_service_client.sync_feature_view(
    feature_view = f"{LOCATION_PARENT}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}"
)

# Start online serving

In [ ]:
data_client = FeatureOnlineStoreServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)

In [ ]:
data_client.fetch_feature_values(
    request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
        feature_view=f"projects/{PROJECT_ID}/locations/{REGION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}/featureViews/{FEATURE_VIEW_ID}",
        data_key=feature_online_store_service_pb2.FeatureViewDataKey(key="test"),
    )
)

## Clean up

Delete the resources created in this tutorial.

In [ ]:
!read -p "Are you sure? " -n 1 -r
feature_online_store_admin_service_client.delete_feature_online_store({
  "name": LOCATION_PARENT + "/featureOnlineStores/" + FEATURE_ONLINE_STORE_ID,
  "force": True
}).result()

# Delete test data
!bq rm -f {DATASET_ID}.{TABLE_ID}